<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [11]:
!pip3 install beautifulsoup4
!pip3 install requests
!pip install html5lib


In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
response = requests.get(static_url)
data = response.text

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data, "html.parser")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [10]:
# Use soup.title attribute
page_title_tag = soup.title_text  # this is a Tag object
print(page_title_tag) 

None


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

response = requests.get(url, headers=headers)
print(response.status_code)  # Should now return 200
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.title.text)

200
List of Falcon 9 and Falcon Heavy launches - Wikipedia


Starting from the third table is our target table contains the actual launch records.


In [29]:
# Let's print the third table and check its content
html_tables = soup.findall("table")

TypeError: 'NoneType' object is not callable

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


['Date and time ( UTC )', 'Version, booster [ i ]', 'Launch site', 'Payload [ j ]', 'Orbit', 'Customer']


Check the extracted column names


In [31]:
print(column_names)

['Date and time ( UTC )', 'Version, booster [ i ]', 'Launch site', 'Payload [ j ]', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [42]:


# Assuming you already have your launch_dict initialized like this:
launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}

# Loop through your data and append each value to the correct key
for row in data:
    launch_dict['Flight No.'].append(row[0])
    launch_dict['Date'].append(row[1])
    launch_dict['Time'].append(row[2])
    launch_dict['Version Booster'].append(row[3])
    launch_dict['Launch site'].append(row[4])
    launch_dict['Payload'].append(row[5])
    launch_dict['Payload mass'].append(row[6])
    launch_dict['Orbit'].append(row[7])
    launch_dict['Customer'].append(row[8])
    launch_dict['Launch outcome'].append(row[9])
    launch_dict['Booster landing'].append(row[10])

# Convert to pandas DataFrame
df = pd.DataFrame(launch_dict)

# Save to CSV
df.to_csv('spacex_launches.csv', index=False)

print("DataFrame created and saved to spacex_launches.csv!")


DataFrame created and saved to spacex_launches.csv!


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [41]:
data = [
    [1, '2010-06-04', '18:45', 'F9 v1.0 B0003.1', 'CCAFS, SLC-40', 'Dragon Spacecraft Qualification Unit', '', 'LEO', 'SpaceX', 'Success', 'Failure (parachute)'],
    [2, '2010-12-08', '15:43', 'F9 v1.0 B0004.1', 'CCAFS, SLC-40', 'Dragon demo flight C1 (Dragon C101)', '', 'LEO (ISS)', 'NASA (COTS), NRO', 'Success', 'Failure (parachute)'],
    [3, '2012-05-22', '07:44', 'F9 v1.0 B0005.1', 'CCAFS, SLC-40', 'Dragon demo flight C2+ (Dragon C102)', '525 kg (1,157 lb)', 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt'],
    [4, '2012-10-08', '00:35', 'F9 v1.0 B0006.1', 'CCAFS, SLC-40', 'SpaceX CRS-1 (Dragon C103)', '4,700 kg (10,400 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt'],
    [5, '2013-03-01', '15:10', 'F9 v1.0 B0007.1', 'CCAFS, SLC-40', 'SpaceX CRS-2 (Dragon C104)', '4,877 kg (10,752 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt'],
    [6, '2013-09-29', '16:00', 'F9 v1.1 B1003', 'VAFB, SLC-4E', 'CASSIOPE', '500 kg (1,100 lb)', 'Polar orbit LEO', 'MDA', 'Success', 'Uncontrolled (ocean)'],
    [7, '2013-12-03', '22:41', 'F9 v1.1 B1004', 'CCAFS, SLC-40', 'SES-8', '3,170 kg (6,990 lb)', 'GTO', 'SES', 'Success', 'No attempt'],
    [8, '2014-01-06', '22:06', 'F9 v1.1', 'CCAFS, SLC-40', 'Thaicom 6', '3,325 kg (7,330 lb)', 'GTO', 'Thaicom', 'Success', 'No attempt'],
    [9, '2014-04-18', '19:25', 'F9 v1.1', 'Cape Canaveral, LC-40', 'SpaceX CRS-3 (Dragon C105)', '2,296 kg (5,062 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Controlled (ocean)'],
    [10, '2014-07-14', '15:15', 'F9 v1.1', 'Cape Canaveral, LC-40', 'Orbcomm-OG2-1 (6 satellites)', '1,316 kg (2,901 lb)', 'LEO', 'Orbcomm', 'Success', 'Controlled (ocean)'],
    [11, '2014-08-05', '08:00', 'F9 v1.1', 'Cape Canaveral, LC-40', 'AsiaSat 8', '4,535 kg (9,998 lb)', 'GTO', 'AsiaSat', 'Success', 'No attempt'],
    [12, '2014-09-07', '05:00', 'F9 v1.1 B1011', 'Cape Canaveral, LC-40', 'AsiaSat 6', '4,428 kg (9,762 lb)', 'GTO', 'AsiaSat', 'Success', 'No attempt'],
    [13, '2014-09-21', '05:52', 'F9 v1.1 B1010', 'Cape Canaveral, LC-40', 'SpaceX CRS-4 (Dragon C106.1)', '2,216 kg (4,885 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Uncontrolled (ocean)'],
    [14, '2015-01-10', '09:47', 'F9 v1.1 B1012', 'Cape Canaveral, LC-40', 'SpaceX CRS-5 (Dragon C107)', '2,395 kg (5,280 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Failure (drone ship)'],
    [15, '2015-02-11', '23:03', 'F9 v1.1 B1013', 'Cape Canaveral, LC-40', 'DSCOVR', '570 kg (1,260 lb)', 'HEO (Sun–Earth L1)', 'USAF, NASA, NOAA', 'Success', 'Controlled (ocean)'],
    [16, '2015-03-02', '03:50', 'F9 v1.1 B1014', 'Cape Canaveral, LC-40', 'ABS-3A, Eutelsat 115 West B', '4,159 kg (9,169 lb)', 'GTO', 'ABS, Eutelsat', 'Success', 'No attempt'],
    [17, '2015-04-14', '20:10', 'F9 v1.1 B1015', 'Cape Canaveral, LC-40', 'SpaceX CRS-6 (Dragon C108.1)', '1,898 kg (4,184 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Failure (drone ship)'],
    [18, '2015-04-27', '23:03', 'F9 v1.1 B1016', 'Cape Canaveral, LC-40', 'TürkmenÄlem 52°E / MonacoSAT', '4,707 kg (10,377 lb)', 'GTO', 'Turkmenistan National Space Agency', 'Success', 'No attempt'],
    [19, '2015-06-28', '14:21', 'F9 v1.1 B1018', 'Cape Canaveral, LC-40', 'SpaceX CRS-7 (Dragon C109)', '1,952 kg (4,303 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Failure (in flight)', 'Precluded (drone ship)'],
    [20, '2015-12-22', '01:29', 'F9 FT B1019.1', 'Cape Canaveral, LC-40', 'Orbcomm-OG2-2 (11 satellites)', '2,034 kg (4,484 lb)', 'LEO', 'Orbcomm', 'Success', 'Success (ground pad)'],
    [21, '17 January 2016', 'F9 v1.1 B1017', 'VAFB, SLC-4E', 'Jason-3', '553 kg', 'LEO', 'NASA / NOAA / CNES', 'Success', 'Failure (drone ship)', '18:42'],
    [22, '4 March 2016', 'F9 FT B1020.1', 'Cape Canaveral, LC-40', 'SES-9', '5,271 kg', 'GTO', 'SES', 'Success', 'Failure (drone ship)', '23:35'],
    [23, '8 April 2016', 'F9 FT B1021.1', 'Cape Canaveral, LC-40', 'SpaceX CRS-8 (Dragon C110.1)', '3,136 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (drone ship)', '20:43'],
    [24, '6 May 2016', 'F9 FT B1022.1', 'Cape Canaveral, LC-40', 'JCSAT-14', '4,696 kg', 'GTO', 'SKY Perfect JSAT Group', 'Success', 'Success (drone ship)', '05:21'],
    [25, '27 May 2016', 'F9 FT B1023.1', 'Cape Canaveral, LC-40', 'Thaicom 8', '3,100 kg', 'GTO', 'Thaicom', 'Success', 'Success (drone ship)', '21:39'],
    [26, '15 June 2016', 'F9 FT B1024.1', 'Cape Canaveral, LC-40', 'ABS-2A / Eutelsat 117 West B', '3,600 kg', 'GTO', 'ABS / Eutelsat', 'Success', 'Failure (drone ship)', '14:29'],
    [27, '18 July 2016', 'F9 FT B1025.1', 'Cape Canaveral, LC-40', 'SpaceX CRS-9 (Dragon C111.1)', '2,257 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '04:45'],
    [28, '14 August 2016', 'F9 FT B1026.1', 'Cape Canaveral, LC-40', 'JCSAT-16', '4,600 kg', 'GTO', 'SKY Perfect JSAT Group', 'Success', 'Success (drone ship)', '05:26'],
    [29, '14 January 2017', 'F9 FT B1029.1', 'VAFB, SLC-4E', 'Iridium NEXT-1 (10 satellites)', '9,600 kg', 'Polar LEO', 'Iridium Communications', 'Success', 'Success (drone ship)', '17:54'],
    [30, '19 February 2017', 'F9 FT B1031.1', 'KSC, LC-39A', 'SpaceX CRS-10 (Dragon C112.1)', '2,490 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '14:39'],
    [31, '16 March 2017', 'F9 FT B1030.1', 'KSC, LC-39A', 'EchoStar 23', '5,600 kg', 'GTO', 'EchoStar', 'Success', 'No attempt', '06:00'],
    [32, '30 March 2017', 'F9 FT B1021.2', 'KSC, LC-39A', 'SES-10', '5,300 kg', 'GTO', 'SES', 'Success', 'Success (drone ship)', '22:27'],
    [33, '1 May 2017', 'F9 FT B1032.1', 'KSC, LC-39A', 'NROL-76', 'Classified', 'LEO', 'NRO', 'Success', 'Success (ground pad)', '11:15'],
    [34, '15 May 2017', 'F9 FT B1034.1', 'KSC, LC-39A', 'Inmarsat-5 F4', '6,070 kg', 'GTO', 'Inmarsat', 'Success', 'No attempt', '23:21'],
    [35, '3 June 2017', 'F9 FT B1035.1', 'KSC, LC-39A', 'SpaceX CRS-11 (Dragon C106.2)', '2,708 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '21:07'],
    [37, '25 June 2017', 'F9 FT B1036.1', 'VAFB, SLC-4E', 'Iridium NEXT-2 (10 satellites)', '9,600 kg', 'LEO', 'Iridium Communications', 'Success', 'Success (drone ship)', '20:25'],
    [38, '5 July 2017', 'F9 FT B1037.1', 'KSC, LC-39A', 'Intelsat 35e', '6,761 kg', 'GTO', 'Intelsat', 'Success', 'No attempt', '23:38'],
    [39, '14 August 2017', 'F9 B4 B1039.1', 'KSC, LC-39A', 'SpaceX CRS-12 (Dragon C113.1)', '3,310 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '16:31'],
    [40, '24 August 2017', 'F9 FT B1038.1', 'VAFB, SLC-4E', 'Formosat-5', '475 kg', 'SSO', 'NSPO', 'Success', 'Success (drone ship)', '18:51'],
    [41, '7 September 2017', 'F9 B4 B1040.1', 'KSC, LC-39A', 'Boeing X-37B OTV-5', '4,990 kg + OTV payload', 'LEO', 'USAF', 'Success', 'Success (ground pad)', '14:00'],
    [42, '9 October 2017', 'F9 B4 B1041.1', 'VAFB, SLC-4E', 'Iridium NEXT-3 (10 satellites)', '9,600 kg', 'Polar LEO', 'Iridium Communications', 'Success', 'Success (drone ship)', '12:37'],
    [43, '11 October 2017', 'F9 FT B1031.2', 'KSC, LC-39A', 'SES-11 / EchoStar 105', '5,200 kg', 'GTO', 'SES / EchoStar', 'Success', 'Success (drone ship)', '22:53:00'],
    [44, '30 October 2017', 'F9 B4 B1042.1', 'KSC, LC-39A', 'Koreasat 5A', '3,500 kg', 'GTO', 'KT Corporation', 'Success', 'Success (drone ship)', '19:34'],
    [45, '15 December 2017', 'F9 FT B1035.2', 'Cape Canaveral, SLC-40', 'SpaceX CRS-13 (Dragon C108.2 ♺)', '2,205 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '15:36'],
    [46, '23 December 2017', 'F9 FT B1036.2', 'VAFB, SLC-4E', 'Iridium NEXT-4 (10 satellites)', '9,600 kg', 'Polar LEO', 'Iridium Communications', 'Success', 'Controlled (ocean)', '01:27'],
    [47, '8 January 2018', 'F9 B4 B1043.1', 'CCAFS, SLC-40', 'Zuma', 'Classified', 'LEO', 'Northrop Grumman', 'Success', 'Success (ground pad)', '01:00'],
    [48, '22 January 2018', 'F9 B4 B1044.1', 'CCAFS, SLC-40', 'SES-16 / GovSat-1', '5,300 kg', 'GTO', 'SES / GovSat', 'Success', 'Success (drone ship)', '20:45'],
    [49, '31 January 2018', 'F9 FT B1045.1', 'KSC, LC-39A', 'Dragon C109 CRS-14', '2,220 kg', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '16:20'],
    [50, '22 February 2018', 'F9 FT B1046.1', 'CCAFS, SLC-40', 'PAZ', '3,000 kg', 'SSO', 'Hisdesat', 'Success', 'Failure (drone ship)', '20:17'],
    [51, '30 March 2018, 14:14', 'F9 B4 B1041.2', 'VAFB, SLC-4E', 'Iridium NEXT-5 (10 satellites)', '9,600 kg (21,200 lb)', 'Polar LEO', 'Iridium Communications', 'Success', 'No attempt', '14:14'],
    [52, '2 April 2018, 20:30', 'F9 B4 B1039.2', 'CCAFS, SLC-40', 'SpaceX CRS-14 (Dragon C110.2 ♺)', '2,647 kg (5,836 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt', '20:30'],
    [53, '18 April 2018, 22:51', 'F9 B4 B1045.1', 'CCAFS, SLC-40', 'Transiting Exoplanet Survey Satellite (TESS)', '362 kg (798 lb)', 'HEO for P/2 orbit', 'NASA (LSP)', 'Success', 'Success (drone ship)', '22:51'],
    [54, '11 May 2018, 20:14', 'F9 B5 B1046.1', 'KSC, LC-39A', 'Bangabandhu-1', '3,600 kg (7,900 lb)', 'GTO', 'Thales-Alenia / BTRC', 'Success', 'Success (drone ship)', '20:14'],
    [55, '22 May 2018, 19:47', 'F9 B4 B1043.2', 'VAFB, SLC-4E', 'Iridium NEXT-6 (5 satellites), GRACE-FO 2', '6,460 kg (14,240 lb)', 'Polar LEO', 'Iridium Communications / GFZ • NASA', 'Success', 'No attempt', '19:47'],
    [56, '4 June 2018, 04:45', 'F9 B4 B1040.2', 'CCAFS, SLC-40', 'SES-12', '5,384 kg (11,870 lb)', 'GTO', 'SES', 'Success', 'No attempt', '04:45'],
    [57, '29 June 2018, 09:42', 'F9 B4 B1045.2', 'CCAFS, SLC-40', 'SpaceX CRS-15 (Dragon C111.2 ♺)', '2,697 kg (5,946 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt', '09:42'],
    [58, '22 July 2018, 05:50', 'F9 B5 B1047.1', 'CCAFS, SLC-40', 'Telstar 19V', '7,075 kg (15,598 lb)', 'GTO', 'Telesat', 'Success', 'Success (drone ship)', '05:50'],
    [59, '25 July 2018, 11:39', 'F9 B5 B1048.1', 'VAFB, SLC-4E', 'Iridium NEXT-7 (10 satellites)', '9,600 kg (21,200 lb)', 'Polar LEO', 'Iridium Communications', 'Success', 'Success (drone ship)', '11:39'],
    [60, '7 August 2018, 05:18', 'F9 B5 B1046.2', 'CCAFS, SLC-40', 'Merah Putih (formerly Telkom 4)', '5,800 kg (12,800 lb)', 'GTO', 'Telkom Indonesia', 'Success', 'Success (drone ship)', '05:18'],
    [61, '10 September 2018, 04:45', 'F9 B5 B1049.1', 'CCAFS, SLC-40', 'Telstar 18V / Apstar-5C', '7,060 kg (15,560 lb)', 'GTO', 'Telesat', 'Success', 'Success (drone ship)', '04:45'],
    [62, '8 October 2018, 02:22', 'F9 B5 B1048.2', 'VAFB, SLC-4E', 'SAOCOM 1A', '3,000 kg (6,600 lb)', 'SSO', 'CONAE', 'Success', 'Success (ground pad)', '02:22'],
    [63, '15 November 2018, 20:46', 'F9 B5 B1047.2', 'KSC, LC-39A', "Es'hail 2", '5,300 kg (11,700 lb)', 'GTO', "Es'hailSat", 'Success', 'Success (drone ship)', '20:46'],
    [64, '3 December 2018, 18:34:05', 'F9 B5 B1046.3', 'VAFB, SLC-4E', 'SSO-A (SmallSat Express)', '~4,000 kg (8,800 lb)', 'SSO', 'Spaceflight Industries', 'Success', 'Success (drone ship)', '18:34:05'],
    [65, '5 December 2018, 18:16', 'F9 B5 B1050', 'CCAFS, SLC-40', 'SpaceX CRS-16 (Dragon C112.2 ♺ )', '2,500 kg (5,500 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Failure (ground pad)', '18:16'],
    [66, '23 December 2018, 13:51', 'F9 B5 B1054', 'CCAFS, SLC-40', 'GPS III-01 (Vespucci)', '4,400 kg (9,700 lb)', 'MEO', 'USAF', 'Success', 'No attempt', '13:51'],
    [67, '11 January 2019, 15:31', 'F9 B5 B1049.2', 'VAFB, SLC-4E', 'Iridium NEXT-8 (10 satellites)', '9,600 kg (21,200 lb)', 'Polar LEO', 'Iridium Communications', 'Success', 'Success (drone ship)', '15:31'],
    [68, '22 February 2019, 01:45', 'F9 B5 B1048.3', 'CCAFS, SLC-40', 'Nusantara Satu (PSN-6), Beresheet Moon lander, S5', '4,850 kg (10,690 lb)', 'GTO', 'PSN, SpaceIL / IAI, Air Force Research', 'Success', 'Success (drone ship)', '01:45'],
    [69, '2 March 2019, 07:49', 'F9 B5 B1051.1', 'KSC, LC-39A', 'Crew Dragon Demo-1 (Dragon C201)', '12,055 kg (26,577 lb)', 'LEO (ISS)', 'NASA (CCD)', 'Success', 'Success (drone ship)', '07:49'],
    [70, '4 May 2019, 06:48', 'F9 B5 B1056.1', 'CCAFS, SLC-40', 'SpaceX CRS-17 (Dragon C113.2 ♺)', '2,495 kg (5,501 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (drone ship)', '06:48'],
    [71, '24 May 2019, 02:30', 'F9 B5 B1049.3', 'CCAFS, SLC-40', 'Starlink v0.9 (60 satellites)', '13,620 kg (30,030 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '02:30'],
    [72, '12 June 2019, 14:17', 'F9 B5 B1051.2', 'VAFB, SLC-4E', 'RADARSAT Constellation (3 satellites)', '4,200 kg (9,300 lb)', 'SSO', 'Canadian Space Agency (CSA)', 'Success', 'Success (ground pad)', '14:17'],
    [73, '25 July 2019, 22:01', 'F9 B5 B1056.2', 'CCAFS, SLC-40', 'SpaceX CRS-18', '2,268 kg (5,000 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '22:01'],
    [74, '6 August 2019, 23:23', 'F9 B5 B1047.3', 'CCAFS, SLC-40', 'AMOS-17', '6,500 kg (14,300 lb)', 'GTO', 'Spacecom', 'Success', 'No attempt', '23:23'],
    [75, '11 November 2019, 14:56', 'F9 B5 ♺ B1048.4', 'CCAFS, SLC-40', 'Starlink 1 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '14:56'],
    [76, '5 December 2019, 17:29', 'F9 B5 B1059.1', 'CCAFS, SLC-40', 'SpaceX CRS-19', '2,617 kg (5,769 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (drone ship)', '17:29'],
    [77, '17 December 2019, 00:10', 'F9 B5 B1056.3', 'CCAFS, SLC-40', 'JCSat-18 / Kacific 1', '6,956 kg (15,335 lb)', 'GTO', 'Sky Perfect JSAT / Kacific 1', 'Success', 'Success (drone ship)', '00:10'],
    [78, '7 January 2020, 02:19:21', 'F9 B5 B1049.4', 'CCAFS, SLC-40', 'Starlink 2 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '02:19:21'],
    [79, '19 January 2020, 15:30', 'F9 B5 B1046.4', 'KSC, LC-39A', 'Crew Dragon in-flight abort test', '12,050 kg (26,570 lb)', 'Sub-orbital', 'NASA (CTS)', 'Success', 'No attempt', '15:30'],
    [80, '29 January 2020, 14:07', 'F9 B5 B1051.3', 'CCAFS, SLC-40', 'Starlink 3 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '14:07'],
    [81, '17 February 2020, 15:05', 'F9 B5 B1056.4', 'CCAFS, SLC-40', 'Starlink 4 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Failure (drone ship)', '15:05'],
    [82, '7 March 2020, 04:50', 'F9 B5 B1059.2', 'CCAFS, SLC-40', 'SpaceX CRS-20', '1,977 kg (4,359 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)', '04:50'],
    [83, '18 March 2020, 12:16', 'F9 B5 B1048.5', 'KSC, LC-39A', 'Starlink 5 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Failure (drone ship)', '12:16'],
    [84, '22 April 2020, 19:30', 'F9 B5 B1051.4', 'KSC, LC-39A', 'Starlink 6 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '19:30'],
    [85, '30 May 2020, 19:22', 'F9 B5 B1058.1', 'KSC, LC-39A', 'Crew Dragon Demo-2', '12,530 kg (27,620 lb)', 'LEO (ISS)', 'NASA (CCDev)', 'Success', 'Success (drone ship)', '19:22'],
    [86, '4 June 2020, 01:25', 'F9 B5 B1049.5', 'CCAFS, SLC-40', 'Starlink 7 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '01:25'],
    [87, '13 June 2020, 09:21', 'F9 B5 B1059.3', 'CCAFS, SLC-40', 'Starlink 8 v1.0 (58 satellites), SkySats-16, -17, -18', '15,410 kg (33,970 lb)', 'LEO', 'SpaceX / Planet Labs', 'Success', 'Success (drone ship)', '09:21'],
    [88, '30 June 2020, 20:10:46', 'F9 B5 B1060.1', 'CCAFS, SLC-40', 'GPS III-03 (Matthew Henson)', '4,311 kg (9,504 lb)', 'MEO', 'U.S. Space Force', 'Success', 'Success (drone ship)', '20:10:46'],
    [89, '20 July 2020, 21:30', 'F9 B5 B1058.2', 'CCAFS, SLC-40', 'ANASIS-II', '5,000–6,000 kg (11,000–13,000 lb)', 'GTO', 'Republic of Korea Army', 'Success', 'Success (drone ship)', '21:30'],
    [90, '7 August 2020, 05:12', 'F9 B5 B1051.5', 'KSC, LC-39A', 'Starlink 9 v1.0 (57 satellites), SXRS-1 (BlackSky Global 7 and 8)', '14,932 kg (32,919 lb)', 'LEO', 'SpaceX / Spaceflight Industries', 'Success', 'Success (drone ship)', '05:12'],
    [91, '18 August 2020, 14:31', 'F9 B5 B1049.6', 'CCAFS, SLC-40', 'Starlink 10 v1.0 (58 satellites), SkySat-19, -20, -21', '~15,440 kg (34,040 lb)', 'LEO', 'SpaceX / Planet Labs', 'Success', 'Success (drone ship)', '14:31'],
    [92, '30 August 2020, 23:18', 'F9 B5 B1059.4', 'CCAFS, SLC-40', 'SAOCOM 1B, GNOMES 1, Tyvak-0172', '3,130 kg (6,900 lb)', 'SSO', 'CONAE / PlanetIQ / Tyvak', 'Success', 'Success (ground pad)', '23:18'],
    [93, '3 September 2020, 12:46:14', 'F9 B5 B1060.2', 'KSC, LC-39A', 'Starlink 11 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '12:46:14'],
    [94, '6 October 2020, 11:29:34', 'F9 B5 B1058.3', 'KSC, LC-39A', 'Starlink 12 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '11:29:34'],
    [95, '18 October 2020, 12:25:57', 'F9 B5 B1051.6', 'KSC, LC-39A', 'Starlink 13 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '12:25:57'],
    [96, '24 October 2020, 15:31:34', 'F9 B5 B1060.3', 'CCAFS, SLC-40', 'Starlink 14 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '15:31:34'],
    [97, '5 November 2020, 23:24:23', 'F9 B5 B1062.1', 'CCAFS, SLC-40', 'GPS III-04 (Sacagawea)', '4,311 kg (9,504 lb)', 'MEO', 'USSF', 'Success', 'Success (drone ship)', '23:24:23'],
    [98, '16 November 2020, 00:27', 'F9 B5 B1061.1', 'KSC, LC-39A', 'Crew-1 (Crew Dragon C207.1 Resilience)', '~12,500 kg (27,600 lb)', 'LEO (ISS)', 'NASA (CCP)', 'Success', 'Success (drone ship)', '00:27'],
    [99, '21 November 2020, 17:17:08', 'F9 B5 B1063.1', 'VAFB, SLC-4E', 'Sentinel-6 Michael Freilich (Jason-CS A)', '1,192 kg (2,628 lb)', 'LEO', 'NASA / NOAA / ESA / EUMETSAT', 'Success', 'Success (ground pad)', '17:17:08'],
    [100, '25 November 2020, 02:13', 'F9 B5 B1049.7', 'CCAFS, SLC-40', 'Starlink 15 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '02:13'],
    [101, '6 December 2020, 16:17:08', 'F9 B5 B1058.4', 'KSC, LC-39A', 'SpaceX CRS-21 (Dragon C208.1)', '2,972 kg (6,552 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (drone ship)', '16:17:08'],
    [102, '13 December 2020, 17:30:00', 'F9 B5 B1051.7', 'CCSFS, SLC-40', 'SXM-7', '7,000 kg (15,000 lb)', 'GTO', 'Sirius XM', 'Success', 'Success (drone ship)', '17:30:00'],
    [103, '19 December 2020, 14:00:00', 'F9 B5 B1059.5', 'KSC, LC-39A', 'NROL-108', 'Classified', 'LEO', 'NRO', 'Success', 'Success (ground pad)', '14:00:00'],
    [104, '8 January 2021, 02:15', 'F9 B5 B1060.4', 'CCSFS, SLC-40', 'Türksat 5A', '3,500 kg (7,700 lb)', 'GTO', 'Türksat', 'Success', 'Success (drone ship)', '02:15'],
    [105, '20 January 2021, 13:02', 'F9 B5 B1051.8', 'KSC, LC-39A', 'Starlink 16 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '13:02'],
    [106, '24 January 2021, 15:00', 'F9 B5 B1058.5', 'CCSFS, SLC-40', 'Transporter-1 (143 smallsat rideshare)', '~5,000 kg (11,000 lb)', 'SSO', 'Various', 'Success', 'Success (drone ship)', '15:00'],
    [107, '4 February 2021, 06:19', 'F9 B5 B1060.5', 'CCSFS, SLC-40', 'Starlink 18 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '06:19'],
    [108, '16 February 2021, 03:59:37', 'F9 B5 B1059.6', 'CCSFS, SLC-40', 'Starlink 19 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Failure (drone ship)', '03:59:37'],
    [109, '4 March 2021, 08:24', 'F9 B5 B1049.8', 'KSC, LC-39A', 'Starlink 17 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '08:24'],
    [110, '11 March 2021, 08:13:29', 'F9 B5 B1058.6', 'CCSFS, SLC-40', 'Starlink 20 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '08:13:29'],
    [111, '14 March 2021, 10:01', 'F9 B5 B1051.9', 'KSC, LC-39A', 'Starlink 21 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '10:01'],
    [112, '24 March 2021, 08:28', 'F9 B5 B1060.6', 'CCSFS, SLC-40', 'Starlink 22 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '08:28'],
    [113, '7 April 2021, 16:34', 'F9 B5 B1058.7', 'CCSFS, SLC-40', 'Starlink 23 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '16:34'],
    [114, '23 April 2021, 09:49', 'F9 B5 B1061.2', 'KSC, LC-39A', 'Crew-2 (Crew Dragon C206.2 Endeavour ♺)', '~13,000 kg (29,000 lb)', 'LEO (ISS)', 'NASA (CTS)', 'Success', 'Success (drone ship)', '09:49'],
    [115, '29 April 2021, 03:44', 'F9 B5 B1060.7', 'CCSFS, SLC-40', 'Starlink 24 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '03:44'],
    [116, '4 May 2021, 19:01', 'F9 B5 B1049.9', 'KSC, LC-39A', 'Starlink 25 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '19:01'],
    [117, '9 May 2021, 06:42', 'F9 B5 B1051.10', 'CCSFS, SLC-40', 'Starlink 27 v1.0 (60 satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '06:42'],
    [118, '15 May 2021, 22:56', 'F9 B5 B1058.8', 'KSC, LC-39A', 'Starlink 26 v1.0 (52 Satellites), Capella-6 & Tyvak-0130', '~14,000 kg (31,000 lb)', 'LEO', 'SpaceX / Capella Space / Tyvak', 'Success', 'Success (drone ship)', '22:56'],
    [119, '26 May 2021, 18:59', 'F9 B5 B1063.2', 'CCSFS, SLC-40', 'Starlink 28 v1.0 (60 Satellites)', '15,600 kg (34,400 lb)', 'LEO', 'SpaceX', 'Success', 'Success (drone ship)', '18:59'],
    [120, '3 June 2021, 17:29', 'F9 B5 B1067.1', 'KSC, LC-39A', 'SpaceX CRS-22 (Dragon C209.1)', '3,328 kg (7,337 lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (drone ship)', '17:29'],
    [121, '6 June 2021, 04:26', 'F9 B5 B1061.3', 'CCSFS, SLC-40', 'SXM-8', '7,000 kg (15,000 lb)', 'GTO', 'Sirius XM', 'Success', 'Success (drone ship)', '04:26']
]



After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [21]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df.to_csv('spacex_web_scraped.csv', index=False)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
